In [1]:
import yfinance as yf
import pandas as pd
import numpy as np

In [2]:
# If you don't have csv
# Download it using 
# df = yf.Ticker('infy.NS').history(period='max', interval="1wk",actions=False)
# df.to_csv("infy_weekly.csv")
df = pd.read_csv("reliance_monthly.csv")
df.reset_index()
df.set_index('Date')

,Open,High,Low,Close,Volume
Date,,,,,
1996-01-01,10.471230,11.081626,7.766588,9.028243,3.587400e+09
1996-02-01,8.938852,12.511840,8.913313,10.821119,6.910292e+09
1996-03-01,10.905402,12.156842,9.832740,10.565725,5.434432e+09
1996-04-01,10.611698,13.408283,10.522310,12.496519,4.130763e+09
1996-05-01,13.638136,13.786266,11.520906,12.174718,5.496793e+09
...,...,...,...,...,...
2022-02-01,2408.000000,2456.399902,2243.000000,2359.550049,1.158835e+08
2022-03-01,2359.550049,2688.000000,2180.000000,2634.750000,1.450252e+08
2022-04-01,2636.000000,2856.149902,2521.800049,2790.250000,1.362090e+08


# Methodology
I will be identifying demand zone structure and supply zone structure on basis of candlestick structure. After identifying demand zone, we can further optimize it by checking its strength, identifying factors such as consolidation and can use it to automate and make our chart analysis process easier.

Boring Candles -> Candles which match the criterion -> Size of body less than 50% of size of wicks
Exciting Candles -> Candles which match the criterion -> Size of body greater than 50% of size of wicks.

Demand zones -> Structure which satisfies this order of candlesticks in given order
-> Red Exciting Candle followed by (1-4) boring candles followed with Strong Green Exciting Candle

![Drop Base Rally Demand Zone](./images/dbr_zone.png)

In [3]:
df['boringCandle'] = (abs(df['Open']-df['Close']) / abs(df['High']-df['Low']) < 0.5)
df['greenExcitingCandle'] = (abs(df['Open']-df['Close'])/abs(df['High']-df['Low']) > 0.5) & (df['Open']<df['Close'])
df['redExcitingCandle'] = (abs(df['Open']-df['Close'])/abs(df['High']-df['Low']) > 0.5) & (df['Open']>df['Close'])

In [4]:
df['ds_dbr_base1'] = df['redExcitingCandle'].shift(2) & df['boringCandle'].shift(1) & df['greenExcitingCandle']
df['ds_rbr_base1'] = df['greenExcitingCandle'].shift(2) & df['boringCandle'].shift(1) & df['greenExcitingCandle']

In [5]:
df_ds_dbr_base1 = df.iloc[df.iloc[np.where(df['ds_dbr_base1'] == True)].index - 1]
df_ds_rbr_base1 = df.iloc[df.iloc[np.where(df['ds_rbr_base1'] == True)].index - 1]

In [6]:
df_ds_dbr_base1['greenBoringCandle'] = df_ds_dbr_base1['Open'] < df_ds_dbr_base1['Close']
df_ds_rbr_base1['greenBoringCandle'] = df_ds_rbr_base1['Open'] < df_ds_rbr_base1['Close']

C:\Users\HP\AppData\Local\Temp/ipykernel_9472/51957953.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ds_dbr_base1['greenBoringCandle'] = df_ds_dbr_base1['Open'] < df_ds_dbr_base1['Close']
C:\Users\HP\AppData\Local\Temp/ipykernel_9472/51957953.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ds_rbr_base1['greenBoringCandle'] = df_ds_rbr_base1['Open'] < df_ds_rbr_base1['Close']


In [7]:
dbrProximalLine = []
dbrDistalLine = []
rbrProximalLine = []
rbrDistalLine = []
for i in range(0,len(df_ds_dbr_base1)):
	df_x = df_ds_dbr_base1.iloc[i]
	if df_x['greenBoringCandle']:
		dbrProximalLine.append(df_x['Close'])
	else:
		dbrProximalLine.append(df_x['Open'])
	dbrDistalLine.append(df_x['Low'])
for i in range(0,len(df_ds_rbr_base1)):
	df_x = df_ds_rbr_base1.iloc[i]
	if df_x['greenBoringCandle']:
		rbrProximalLine.append(df_x['Close'])
	else:
		rbrProximalLine.append(df_x['Open'])
	rbrDistalLine.append(df_x['Low'])
df_ds_dbr_base1['proximalLine'] = dbrProximalLine
df_ds_dbr_base1['distalLine'] = dbrDistalLine
df_ds_rbr_base1['proximalLine'] = rbrProximalLine
df_ds_rbr_base1['distalLine'] = rbrDistalLine


C:\Users\HP\AppData\Local\Temp/ipykernel_9472/1002121531.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ds_dbr_base1['proximalLine'] = dbrProximalLine
C:\Users\HP\AppData\Local\Temp/ipykernel_9472/1002121531.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ds_dbr_base1['distalLine'] = dbrDistalLine
C:\Users\HP\AppData\Local\Temp/ipykernel_9472/1002121531.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

In [8]:
df_ds_rbr_base1

,Date,Open,High,Low,Close,Volume,boringCandle,greenExcitingCandle,redExcitingCandle,ds_dbr_base1,ds_rbr_base1,greenBoringCandle,proximalLine,distalLine
2,1996-03-01,10.905402,12.156842,9.832740,10.565725,5.434432e+09,True,False,False,False,False,False,10.905402,9.832740
65,2000-12-01,39.517879,42.635805,39.155332,40.962036,4.944678e+08,True,False,False,False,False,True,40.962036,39.155332
113,2004-08-01,65.155665,74.101220,59.881798,63.506748,8.537769e+08,True,False,False,False,False,False,65.155665,59.881798
132,2006-02-01,156.168079,158.298835,149.371522,154.911484,3.998254e+08,True,False,False,False,False,False,156.168079,149.371522
139,2006-09-01,246.251998,264.317831,236.998216,258.785461,2.910307e+08,True,False,False,False,False,True,258.785461,236.998216
149,2007-06-01,394.812483,400.870072,366.818395,378.721985,1.835756e+08,True,False,False,False,False,False,394.812483,366.818395
151,2007-08-01,414.677791,438.729984,374.858029,436.703369,2.776361e+08,True,False,False,False,False,True,436.703369,374.858029
236,2014-04-01,437.973344,462.656148,434.744696,437.879761,1.194133e+08,True,False,False,False,False,False,437.973344,434.744696
272,2017-01-01,523.164346,529.679737,488.801493,504.438538,1.250663e+08,True,False,False,False,False,False,523.164346,488.801493
297,2018-12-01,1149.430469,1149.430469,1032.129634,1096.943481,1.541826e+08,True,False,False,False,False,False,1149.430469,1032.129634


In [9]:
df_ds_dbr_base1.to_csv('reliance_ds_dbr_monthly.csv')
df_ds_rbr_base1.to_csv('reliance_ds_rbr_monthly.csv')

In [10]:
import plotly.graph_objects as go
fig = go.Figure(data=go.Candlestick(x=df['Date'], open=df['Open'], close=df['Close'], low=df['Low'], high=df['High']))
for row in df_ds_dbr_base1.iterrows():
	fig.add_shape(type="rect",line=dict(color="green", width=2),x0=row[1]['Date'], y0=row[1]['proximalLine'], x1=df.Date.max(), y1=row[1]['distalLine'], fillcolor='green', opacity=0.5)
for row in df_ds_rbr_base1.iterrows():
	fig.add_shape(type="rect",line=dict(color="green", width=2),x0=row[1]['Date'], y0=row[1]['proximalLine'], x1=df.Date.max(), y1=row[1]['distalLine'], fillcolor='green', opacity=0.5)
fig.update_layout(margin={"t":25, "b":0, "l":0, "r":2})
fig.write_html("output/reliance_monthly.html")

In [203]:
# df['proximalLine'].merge(df_ds_dbr_base1['proximalLine'], df_ds_rbr_base1['proximalLine'])

AttributeError: 'Series' object has no attribute 'merge'

In [202]:
# df['proximalLine'][~df['proximalLine'].isna()]

13         0.671573
47         0.941296
49         0.928512
67         1.628934
79         2.503390
92         3.794896
128        6.229283
135        6.002590
178       14.694616
202       41.285553
209       43.602337
212       51.945328
214       54.847088
221       85.808907
240       86.255310
260       78.275520
318       42.523720
356       35.191936
368       43.953907
403       36.873136
425       55.013215
460       66.819774
465       70.766579
473       81.422481
502       92.441064
509      105.169273
661      157.039246
708      116.202950
717      144.921326
719      162.484928
735      211.497421
759      244.420105
790      288.377899
843      226.560392
845      236.961391
847      247.796982
865      278.606068
914      279.868164
939      250.719101
941      289.149475
1070     468.864136
1074     487.488762
1091     517.553103
1133     441.148590
1135     446.833191
1145     421.318005
1173     437.102356
1201     554.171772
1213     629.790411
1229     626.374207
